In [1]:
import os
from time import time
import numpy as np
from PIL import Image
np.set_printoptions(suppress=True)

In [2]:
start_time = time()

In [3]:
consider_distort = False
num_shots = 1

In [4]:
folder = "data/stars/2016nov/" # data folder
res_folder = 'results/2016nov/' # results folder
date = "20161121-220921-250"

In [5]:
# for ipynb
fname_left = folder + os.sep + "mod_" + date + '-1.png'
fname_right = folder + os.sep + "mod_" + date + '-2.png'

# for cmd line run
# fname_left = os.path.abspath(sys.argv[0])
# fname_right = os.path.abspath(sys.argv[1])

In [6]:
img_left = Image.open(fname_left).convert(mode='RGB')
img_right = Image.open(fname_right).convert(mode='RGB')
print """Images loaded as RGB:
%s
%s""" % (fname_left, fname_right)

Images loaded as RGB:
data/stars/2016nov//mod_20161121-220921-250-1.png
data/stars/2016nov//mod_20161121-220921-250-2.png


In [7]:
assert (img_left.size == img_right.size)
w, h = width, height = img_left.size
print 'Original width, height:', w, h

Original width, height: 3112 2328


In [8]:
x_c = img_left.width / 2.0
y_c = img_left.height / 2.0
print 'x_c, y_c:', x_c, y_c

x_c, y_c: 1556.0 1164.0


In [9]:
# Load pixmaps
l_pix = img_left.load()
r_pix = img_right.load()

In [10]:
for i in xrange(width):
    for j in xrange(height):
        r, g, b = l_pix[i, j] # color img_left in red
        l_pix[i, j] = r, 0, 0
        r, g, b = r_pix[i, j]
        r_pix[i, j] = 0, g, 0 # color img_right in green

In [11]:
l_pix[x_c, y_c] = 255, 255, 255    # Mark the center
l_pix[x_c-1, y_c] = 255, 255, 255
l_pix[x_c+1, y_c] = 255, 255, 255
l_pix[x_c, y_c-1] = 255, 255, 255
l_pix[x_c, y_c+1] = 255, 255, 255

In [12]:
align_coeffs = np.loadtxt(res_folder + 'affine_coeffs_shots'+ \
                          str(num_shots) + '_' + date + '.txt')

# HACK! TMP!
# align_coeffs = np.array([1,0,0,1, 240, 39], dtype='float')
# align_coeffs[-2] += 8
# align_coeffs[-1] += 18
np.set_printoptions(precision=12)
print 'Align coeeficients:\n', align_coeffs

Align coeeficients:
[   0.9929384443    0.0081163571   -0.0096512692    1.0092245819
  244.1666666667   37.6666666667]


In [13]:
a = align_coeffs[0];
b = align_coeffs[1];
c = align_coeffs[2];
d = align_coeffs[3];
e = align_coeffs[4];
f = align_coeffs[5];

det = a * d - b * c;
inv_a = d / det;
inv_b = -b / det;
inv_c = -c / det;
inv_d = a / det;

In [14]:
def affine_transform_point(x, y):
    return [b * y + x * a + e , d * y + x * c + f]

In [15]:
def apply_affine(img_left, img_right):
    width = img_left.width
    height = img_left.height
    
    aff_coord = np.zeros((4, 2))
#     affine transformation of the corner points
    aff_coord[0] = affine_transform_point(0, 0)
    aff_coord[1] = affine_transform_point(width, 0)
    aff_coord[2] = affine_transform_point(0, height)
    aff_coord[3] = affine_transform_point(width, height)
    
#     the rightmost (biggest by value) x-coordinate of the transformed
#     left-top and left-bottom x-coordinates
    x0 = int( max(aff_coord[0, 0], aff_coord[2, 0]) )
#     the lowermost (biggest by value) y-coordinate of the transformed
#     left-top and right-top y-coordinates
    y0 = int( max(aff_coord[0, 1], aff_coord[1, 1]) )
#     the leftmost (smallest by value) x-coordinate of the transformed
#     right-top and right-bottom x-coordinates
    x1 = int( min(aff_coord[1, 0], aff_coord[3, 0]) )
#     the uppermost (smallest by value) y-coordinate of the transformed
#     left-bottom and right-bottom y-coordinates
    y1 = int( min(aff_coord[2, 1], aff_coord[3, 1]) )
    
#     n_x0 -- x-coordinate of the new left-bot point
    n_x0 = int( max(0, x0) )
#     n_y0 -- y-coordinate of the new left-bot point
    n_y0 = int( max(0, y0) )
#     n_x1 -- x-coordinate of the new right-top point
    n_x1 = int( min(width, x1) )
#     n_y1 -- y-coordinate of the new right-top point
    n_y1 = int( min(height, y1) )
    
    nw = n_x1 - n_x0 # new width
    nh = n_y1 - n_y0 # new height
    
    new_left_img = Image.new(mode='RGB', size=(nw, nh))
    new_right_img = Image.new(mode='RGB', size=(nw, nh))
    
    # Load pixmaps
    l_pix = img_left.load()
    r_pix = img_right.load()
    nl_pix = new_left_img.load()
    nr_pix = new_right_img.load()
    
    
    
    for  y in xrange(n_y0, n_y1):
        for x in xrange(n_x0, n_x1):
# Let's calculate backwards our original coordinates of the left image
            orig_x = int( (x - e) * inv_a + (y - f) * inv_b )
            orig_y = int( (x - e) * inv_c + (y - f) * inv_d )
            
#             assert(0 <= orig_x <= width)
#             assert(0 <= orig_y <= height)
            
# paint new images with coordinates from (0,0) to (nw - 1, nh - 1)
            nl_pix[x - n_x0, y - n_y0] = l_pix[orig_x, orig_y]
            nr_pix[x - n_x0, y - n_y0] = r_pix[x, y]
    
    return (new_left_img, new_right_img)

In [16]:
width = img_left.width
height = img_left.height

aff_coord = np.zeros((4, 2))
#     affine transformation of the corner points
aff_coord[0] = affine_transform_point(0, 0)
aff_coord[1] = affine_transform_point(width, 0)
aff_coord[2] = affine_transform_point(0, height)
aff_coord[3] = affine_transform_point(width, height)

#     the rightmost (biggest by value) x-coordinate of the transformed
#     left-top and left-bottom x-coordinates
x0 = int( max(aff_coord[0, 0], aff_coord[2, 0]) )
#     the lowermost (biggest by value) y-coordinate of the transformed
#     left-top and right-top y-coordinates
y0 = int( max(aff_coord[0, 1], aff_coord[1, 1]) )
#     the leftmost (smallest by value) x-coordinate of the transformed
#     right-top and right-bottom x-coordinates
x1 = int( min(aff_coord[1, 0], aff_coord[3, 0]) )
#     the uppermost (smallest by value) y-coordinate of the transformed
#     left-bottom and right-bottom y-coordinates
y1 = int( min(aff_coord[2, 1], aff_coord[3, 1]) )

#     n_x0 -- x-coordinate of the new left-bot point
n_x0 = int( max(0, x0) )
#     n_y0 -- y-coordinate of the new left-bot point
n_y0 = int( max(0, y0) )
#     n_x1 -- x-coordinate of the new right-top point
n_x1 = int( min(width, x1) )
#     n_y1 -- y-coordinate of the new right-top point
n_y1 = int( min(height, y1) )

nw = n_x1 - n_x0 # new width
nh = n_y1 - n_y0 # new height

In [17]:
z = align_coeffs

a = float(z[0])
b = float(z[1])
c = float(z[2])
d = float(z[3])
e = float(z[4])
f = float(z[5])
if consider_distort:
    eps1 = float(z[6])
    eps2 = float(z[7])
data = (a, b, e, c, d, f) # affine coefficients

In [18]:
img_left_n = Image.new('RGB', img_left.size, 'black')
img_right_n = Image.new('RGB', img_right.size, 'black')
nl_pix = img_left_n.load()
nr_pix = img_right_n.load()

In [19]:
if consider_distort:
    # Get rid of distortions on nlImg
    
    for i in xrange(width): # xi
        for j in xrange(height): # eta
            dist_l = (i-x_c)**2 + (j-y_c)**2

            zx1 = (i - x_c) * dist_l
            zy1 = (j - y_c) * dist_l

            nl_pix[round(i - eps1*zx1), round(j - eps2*zy1)] = l_pix[i, j]
    
    # Get rid of distortions on nrImg

    for i in xrange(width): # xi
        for j in xrange(height): # eta
            dist_r = (i-x_c)**2 + (j-y_c)**2

            zx2 = (i - x_c) * dist_r
            zy2 = (j - y_c) * dist_r

            nr_pix[round(i - eps1*zx2), round(j - eps2*zy2)] = r_pix[i, j]


In [20]:
# img_left_n, img_right_n = apply_affine(img_left, img_right)
img_left_n = img_left
img_right_n = img_right.transform(img_right.size, Image.AFFINE , 
                                data=(a,b,e,c,d,f), resample=Image.BICUBIC)

In [21]:
print "New width, height:", img_left_n.size

New width, height: (3112, 2328)


In [22]:
Image.blend(img_left, img_right, alpha=0.5).save(res_folder + date[:-4] + '_blend_init' + ".jpg")
if consider_distort:
    Image.blend(img_left_n, img_right_n, alpha=0.5).save(res_folder + date[:-4] + '_blend_dist' + ".jpg")
else:
    Image.blend(img_left_n, img_right_n, alpha=0.5).save(res_folder + date[:-4] + '_blend_aff' + ".jpg")

In [23]:
print "Script running time:", time() - start_time

Script running time: 29.7935979366


In [24]:
print align_coeffs

[   0.9929384443    0.0081163571   -0.0096512692    1.0092245819
  244.1666666667   37.6666666667]
